In [2]:
import ee
import os
import math
from datetime import datetime
# from src.utils.treatment_calibration import get_dials
from src.utils import ee_treatments
from src.utils.yml_params import get_export_params
yml_file = os.path.join(os.getcwd(),'config.yml')
CRS,EXPORT_SCALE = get_export_params(yml_file)

ee.Initialize(project='mas-gee')
%load_ext autoreload
%autoreload 2

# In development phase
tag = 'ee_treatments_export'
ee.data.setWorkloadTag(tag)

## Million Acres Strategy - Treatment Simulator
#### Creates Randomized Treatment Scenario Landscapes - exports as ee.Images
#### This Notebook is for Run IDs: 
|RUNID | CA_REGION | INTENSITY_ID | PRIORITY |
|------|-----------|--------------|----------|
| 1  | Sierra Nevada | 500k | WUI |
| 4  | Sierra Nevada | 1m   | WUI |
| 7  | Sierra Nevada | 2m   | WUI |
| 10 | North Coast   | 500k | WUI |
| 13 | North Coast   | 1m   | WUI |
| 16 | North Coast   | 2m   | WUI |


Run Settings: Change `RUNID`, `CA_REGION`, `INTENSITY_ID`, and `PRIORITY` below to setup the analysis and output file names

In [9]:
# this could be even less manual if CA_REGION, INTENSITY_ID, and PRIORITY were accessed from master_dict below using RUNID..
RUNID = '13' 
CA_REGION = 'NC' # NC = North Coast; SN = Sierra Nevada
INTENSITY_ID = '1m' 
PRIORITY = 'WUI' 

# for each scenario, make the treated landscape raster 
today_string = datetime.utcnow().strftime("%Y-%m-%d").replace("-", "")
output_image = f"projects/pyregence-ee/assets/mas/treatment_scenarios/RunID{RUNID}/{CA_REGION}_{PRIORITY}_{INTENSITY_ID}_{today_string}"
print(f'Output EE Image Path: {output_image}')

Output EE Image Path: projects/pyregence-ee/assets/mas/treatment_scenarios/RunID13/NC_WUI_1m_20230623


Input Files & Global Variables

In [10]:
# Sierra Nevada(SN), South Coast(SC), Central Coast(CC), North Coast(NC)
all_hucs = ee.FeatureCollection("projects/pyregence-ee/assets/mas/TxPrctRankRrkWip")

# Treatable Vegetation Constraint Layer
# For reference Herb (1), Shrub (2), Hardwood (3), softwood (4), non treatable veg is masked
veg_constraint = ee.Image("projects/pyregence-ee/assets/mas/HerbShrubHardSoftImage")

# all area values must be converted to meters for GEE functions
# we will also report things in Ac for double-checking our math
ac_to_sqm_x = 4046.8564224
sqm_to_ac_x = 0.000247105

# Each Region's Total Treatable Vegetation Acres
# from https://docs.google.com/spreadsheets/d/1Gnl6SO5kOkj4Ne1JdXzW4bp03824Zb1Cp_2HVuCY3Lk/edit#gid=0
sn_area = int(round(21979145 * ac_to_sqm_x))
sc_area = int(round(8208838 * ac_to_sqm_x))
cc_area = int(round(7368439 * ac_to_sqm_x))
nc_area = int(round(14716059 * ac_to_sqm_x))

# acreages to treat per year per region per scenario
# from https://docs.google.com/spreadsheets/d/1Gnl6SO5kOkj4Ne1JdXzW4bp03824Zb1Cp_2HVuCY3Lk/edit#gid=0
master_dict = {
    'SN':{'scenarioAcreages':{'500k':210000*ac_to_sqm_x,'1m':420000*ac_to_sqm_x,'2m':840000*ac_to_sqm_x}, 
          'RRK Name': 'Sierra Nevada Region'},
    'SC':{'scenarioAcreages':{'500k':80000*ac_to_sqm_x,'1m':160000*ac_to_sqm_x,'2m':320000*ac_to_sqm_x}, 
          'RRK Name': 'South Coast Region'},
    'CC':{'scenarioAcreages':{'500k':70000*ac_to_sqm_x,'1m':140000*ac_to_sqm_x,'2m':280000*ac_to_sqm_x}, 
          'RRK Name': 'Central Coast Region'},
    'NC':{'scenarioAcreages':{'500k':140000*ac_to_sqm_x,'1m':280000*ac_to_sqm_x,'2m':560000*ac_to_sqm_x}, 
          'RRK Name': 'North Coast Region'},
}

treatment_size = 100 # acres
TREATMENT_SIZE = treatment_size*ac_to_sqm_x # acres to m^2
print('TREATMENT SIZE (m^2):',TREATMENT_SIZE)
print('TREATMENT SIZE (Ac):',treatment_size)
RADIUS = math.sqrt(TREATMENT_SIZE)/2  # in meters, rough square radius is A = (side/2) solve for side (side = sqrt(A) )

TREATMENT SIZE (m^2): 404685.64224
TREATMENT SIZE (Ac): 100


Auto-calculate the necessary information for the Scenario, relies on your variables set in the first code cell at top.

In [11]:
# RRK Region Name
rrk_region = master_dict[CA_REGION]['RRK Name']
print(f'RRK Region: {rrk_region}')

# Develop the prescribed treatment amount
total_treated_area = int(master_dict[CA_REGION]['scenarioAcreages'][INTENSITY_ID])
print(f"Area to Treat Per Year (m^2): {total_treated_area}")
total_treated_area_ac = int(round(total_treated_area*sqm_to_ac_x))
print(f"Area to Treat Per Year (Ac): {total_treated_area_ac}")

# we run the treatment generator as 5-year interval outputs (4 run intervals covering the 20-year period)
# so we actually need to plug in the total area to treat per iteration (which is total_treated_area*5 )
area_per_iteration = int(round(total_treated_area*5))
print(f'Area to treat per 5-year interval (m^2): {area_per_iteration}')
area_per_iteration_ac = int(round(area_per_iteration*sqm_to_ac_x))
print(f'Area to treat per 5-year interval (Ac): {area_per_iteration_ac}')

# For North Coast and Sierra Nevada WUI scenarios, we will treat the 25th, 50th and 75th percentiles of WUI rankings, 
# then re-treat 25th in the last year
year_interval_ids = ee.List(['Y1to5',
                             'Y6to10',
                             'Y11to15',
                             'Y16-20'
                             ]
                             )
huc_filters = [ee.Filter.eq('TxWuiPrctl', 25),
               ee.Filter.eq('TxWuiPrctl', 50),
               ee.Filter.eq('TxWuiPrctl', 75),
               ee.Filter.eq('TxWuiPrctl', 25)
               ]
zipped = year_interval_ids.zip(huc_filters)

RRK Region: North Coast Region
Area to Treat Per Year (m^2): 1133119798
Area to Treat Per Year (Ac): 280000
Area to treat per 5-year interval (m^2): 5665598990
Area to treat per 5-year interval (Ac): 1399998


Necessary functions that help us parallelize the analysis, Run and continue

In [12]:
def treat_by_year_huc(l):
    zipped_list = ee.List(l)
    filter = ee.Filter(zipped_list.get(1))
    # Get HUC group
    huc_group = (all_hucs.filter(ee.Filter.eq('RRK_Region', rrk_region))
                .filter(filter)
                # .limit(2) # for testing, reduce compute for development
                ) 
    
    # area_per_iteration is the prescribed area 
    # area_per_iteration_testing = area_per_iteration*(2/193) 
    
    pixel_value_iteration = ee.Number(ee.List(zipped).indexOf(l)).add(1)
    
    # this is for reshuffling the random treatment placements, may be useful for re-treatments of the same huc groups
    # we would pass a unique number to 'seed' arg in ee_treatments() each time 
    # if you don't need to reshuffle the random seed each run, don't define 'seed' arg and function will use its default seed every time
    seeds = ee.List([10,20,30,40])
    seed = ee.Number(ee.List(seeds).get(ee.List(zipped).indexOf(l)))
    
    treated_area_img, properties = ee_treatments.ee_treatments(hucs=huc_group,
                                                  prescription=area_per_iteration, # area_per_iteration_testing , reduce compute for testing
                                                  unit_size=TREATMENT_SIZE,
                                                  radius=RADIUS,
                                                  pixel_value=pixel_value_iteration,
                                                  constraint_layer=veg_constraint,
                                                  # seed=seed # passing seed defined as diff number for every ee_treatment() run to randomize re-treatment groups
                                                  )
    return treated_area_img,properties

def remap_keys(d):
  # solves a QA property problem for ee_treatments(): we have duplicate key names in the properties dict 
  # when multiple ee_treatments() results are combined together
  # so we create a unique identifier from yearInterval and remove yearInterval as a property itself, 
  # remapped k,v pairs with yearInterval unique identifiers
  # example: 'PropertyName': PropertyValues becomes 'yearInterval1PropertyName': PropertyValue
  d = ee.Dictionary(d)
  yearInterval = (ee.String('yearInterval')
  .cat(ee.Number(ee.Dictionary(d).get('yearInterval')).format()))
  keys = d.keys().slice(0,7) # depends on yearInterval property being in index position 7 of property list returned by ee_treatments()
  values = d.values().slice(0,7)
  remapped_keys = keys.map(lambda s: ee.String(yearInterval).cat(s))
  return ee.Dictionary.fromLists(remapped_keys,values)

def combine_dicts(d):
   d = ee.Dictionary(d)
   return d.keys().zip(d.values())

Create treated area ee.Images for every year interval on its assigned group of HUCs

In [13]:
# We use .map() to parallelize computation of treatment area images across the four 5-year intervals, 
# treating equal amount of area within each percentile ranking list, returning 4 treatment ee.Images, 
# then mosaicking them into one ee.Image for export

# returns ee.List(tuple1(ee.Image1,ee.Dictionary1),tuple2(ee.Image2,ee.Dictionary2),tuple3(etc))
results = ee.List(zipped).map(treat_by_year_huc) 

# parse the images and properties out of the (image,properties) tuples by .map()ing over results
images = results.map(lambda t: ee.List(t).get(0)) 
properties = results.map(lambda t: ee.List(t).get(1))

# remap the keys of the properties so they are unique per ee_treatments() image run, then combine into one properties dict
all_properties = properties.map(remap_keys).map(combine_dicts).flatten()
# print(all_properties.getInfo())

# mosaic all ee_treatment() images together and set the fixed properties 
output = ee.Image(ee.ImageCollection.fromImages(images).mosaic()).set(all_properties)

# export
run_id = f'RUNID{RUNID}'
desc = f'{run_id}_{os.path.basename(output_image)}'
ee_treatments.export_img(output,desc,output_image,all_hucs.geometry(),30,'EPSG:5070')

Export Started for projects/pyregence-ee/assets/mas/treatment_scenarios/RunID13/NC_WUI_1m_20230623
